# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import warnings
warnings.filterwarnings('ignore')

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Weather Description
0,0,west island,-12.1568,96.8225,26.99,74,40,4.63,CC,scattered clouds
1,1,port elizabeth,-33.9180,25.5701,16.29,94,75,0.00,ZA,broken clouds
2,2,edinburgh of the seven seas,-37.0676,-12.3116,13.09,69,91,5.56,SH,overcast clouds
3,3,puerto natales,-51.7236,-72.4875,7.22,91,100,1.40,CL,light rain
4,4,papatowai,-46.5619,169.4708,13.14,86,91,8.91,NZ,overcast clouds


In [3]:
city_data_df.dropna(inplace=True)
city_data_df.columns

Index(['City_ID', 'City', 'Lat', 'Lng', 'Max Temp', 'Humidity', 'Cloudiness',
       'Wind Speed', 'Country', 'Weather Description'],
      dtype='object')

---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [4]:
# Configure the map plot
map_plot1 = city_data_df.hvplot.points(
        'Lng',
        'Lat', 
        geo=True, 
        scale=1,
        alpha=0.5,
        tiles='OSM',
        frame_width=700,
        frame_height=500,
        size='Humidity'
)

# Display the map
map_plot1

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (Humidity)

In [5]:
city_data_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Weather Description
0,0,west island,-12.1568,96.8225,26.99,74,40,4.63,CC,scattered clouds
1,1,port elizabeth,-33.9180,25.5701,16.29,94,75,0.00,ZA,broken clouds
2,2,edinburgh of the seven seas,-37.0676,-12.3116,13.09,69,91,5.56,SH,overcast clouds
3,3,puerto natales,-51.7236,-72.4875,7.22,91,100,1.40,CL,light rain
4,4,papatowai,-46.5619,169.4708,13.14,86,91,8.91,NZ,overcast clouds
...,...,...,...,...,...,...,...,...,...,...
577,577,ardestan,33.3761,52.3694,13.06,36,7,2.61,IR,clear sky
578,578,nikolayevsk-on-amure,53.1406,140.7300,-14.49,64,5,1.91,RU,clear sky
579,579,la ronge,55.1001,-105.2842,-5.86,86,100,6.17,CA,light snow
580,580,puerto real,36.5282,-6.1901,11.09,82,0,2.57,ES,clear sky


### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [6]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_weather_df = city_data_df.loc[(city_data_df["Max Temp"] < 25) & (city_data_df["Max Temp"] > 20)]

# Drop any rows with null values
cleaned_df = ideal_weather_df.dropna()

# Display sample data
cleaned_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Weather Description
6,6,pisco,-13.7000,-76.2167,20.03,77,0,6.69,PE,clear sky
8,8,adamstown,-25.0660,-130.1015,23.07,86,100,10.25,PN,light rain
9,9,richards bay,-28.7830,32.0377,23.29,95,100,1.13,ZA,overcast clouds
40,40,puerto baquerizo moreno,-0.9000,-89.6000,24.77,80,100,6.04,EC,overcast clouds
41,41,maxixe,-23.8597,35.3472,22.23,89,2,1.71,MZ,clear sky
...,...,...,...,...,...,...,...,...,...,...
566,566,sittwe,20.1500,92.9000,22.38,75,53,3.11,MM,broken clouds
567,567,jacobina,-11.1806,-40.5183,23.08,43,0,3.54,BR,clear sky
571,571,bardai,11.6500,7.2333,24.42,25,100,3.07,NG,overcast clouds
573,573,east ballina,-28.8667,153.5833,20.92,94,100,1.04,AU,overcast clouds


### Step 3: Create a new DataFrame called `hotel_df`.

In [7]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = cleaned_df.copy()
cleaned_hotel_df = hotel_df[["City", "Country","Lat", "Lng", "Humidity"]] 

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
cleaned_hotel_df['Hotel Name']= ''

# Display sample data
cleaned_hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
6,pisco,PE,-13.7000,-76.2167,77,
8,adamstown,PN,-25.0660,-130.1015,86,
9,richards bay,ZA,-28.7830,32.0377,95,
40,puerto baquerizo moreno,EC,-0.9000,-89.6000,80,
41,maxixe,MZ,-23.8597,35.3472,89,
...,...,...,...,...,...,...
566,sittwe,MM,20.1500,92.9000,75,
567,jacobina,BR,-11.1806,-40.5183,43,
571,bardai,NG,11.6500,7.2333,25,
573,east ballina,AU,-28.8667,153.5833,94,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [19]:
# Set parameters to search for a hotel

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude= row['Lat']
    #hotel_df.loc[index, 'Lat']
    longitude= row['Lng'] 
    #hotel_df.loc[index, 'Lng']
    
    radius = 10000
    categories = 'accommodation.hotel'
    limit = 20
    bias = f"proximity:{longitude},{latitude}"
    
    params = {
        "categories":categories,
        "limit":limit,
        "bias":bias,
        "apiKey":geoapify_key
        }
    
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    #print(params)
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params).json()
    #print(name_address)
    
    # Convert the API response to JSON format
    #name_address = # YOUR CODE HERE; not already done above? 
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df
  

Starting hotel search
pisco - nearest hotel: La Portada
adamstown - nearest hotel: No hotel found
richards bay - nearest hotel: SUN1 Richards Bay
puerto baquerizo moreno - nearest hotel: Dory's House
maxixe - nearest hotel: Hotel Herry
nova vicosa - nearest hotel: Pousada Pontal da Barra
teknaf - nearest hotel: No hotel found
bende - nearest hotel: No hotel found
bangassou - nearest hotel: Hotel De Tourisme
kurmuk - nearest hotel: No hotel found
wanning - nearest hotel: No hotel found
tsiombe - nearest hotel: No hotel found
yung shue wan - nearest hotel: 浪濤軒酒店 Concerto Inn
arraial do cabo - nearest hotel: No hotel found
hasaki - nearest hotel: 大新旅館
puerto ayora - nearest hotel: Hostal La Mirada De Solitario George
port mathurin - nearest hotel: Escale Vacances
djibouti - nearest hotel: No hotel found
antalaha - nearest hotel: Hôtel Florida
akjoujt - nearest hotel: فندق فاطمة
nova sintra - nearest hotel: Residência Ka Dencho
whakatane - nearest hotel: Whakatane Hotel
erumaippatti - near

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Weather Description,Hotel Name
6,6,pisco,-13.7000,-76.2167,20.03,77,0,6.69,PE,clear sky,La Portada
8,8,adamstown,-25.0660,-130.1015,23.07,86,100,10.25,PN,light rain,No hotel found
9,9,richards bay,-28.7830,32.0377,23.29,95,100,1.13,ZA,overcast clouds,SUN1 Richards Bay
40,40,puerto baquerizo moreno,-0.9000,-89.6000,24.77,80,100,6.04,EC,overcast clouds,Dory's House
41,41,maxixe,-23.8597,35.3472,22.23,89,2,1.71,MZ,clear sky,Hotel Herry
...,...,...,...,...,...,...,...,...,...,...,...
566,566,sittwe,20.1500,92.9000,22.38,75,53,3.11,MM,broken clouds,Yuzana Aung Motel 1
567,567,jacobina,-11.1806,-40.5183,23.08,43,0,3.54,BR,clear sky,Hotel Jovem Jota
571,571,bardai,11.6500,7.2333,24.42,25,100,3.07,NG,overcast clouds,No hotel found
573,573,east ballina,-28.8667,153.5833,20.92,94,100,1.04,AU,overcast clouds,Ballina Beach Resort


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [20]:
%%capture --no-display

# Configure the map plot
map_plot2 = hotel_df.hvplot.points(
        'Lng',
        'Lat',
        geo=True, 
        scale=1,
        alpha=0.5,
        tiles='OSM',
        frame_width=700,
        frame_heigth=500,
        hover=["Hotel Name","Country"]
        
)

# Display the map
map_plot2

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]